In [ ]:
import numpy as np

# Given points
points = np.array([
    (1,1), (3,1), (2,2), (1,3), (3,3), (7,1), (9,1), (8,2), (7,3), (9,3), (4,7), (6,7), (5,8), (4,9), (6,9)
])

# Step 1: Initialize S by finding the object with the minimal sum of distances
min_index = 0
min_distance_sum = float('inf')

for i in range(len(points)):
    distance_sum = sum(np.sqrt(np.sum((points[i] - points[j]) ** 2)) for j in range(len(points)) if j != i)
    if distance_sum < min_distance_sum:
        min_distance_sum = distance_sum
        min_index = i

# S (selected medoids) and U (unselected objects)
S = {min_index}
U = {i for i in range(len(points)) if i != min_index}

print(f"Initial medoid index: {min_index}, Medoid point: {points[min_index]}")
print(f"Set S (medoids): {S}")
print(f"Set U (unselected objects): {U}")


Initial medoid index: 4, Medoid point: [3 3]
Set S (medoids): {4}
Set U (unselected objects): {0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}


In [ ]:
# Step 2: Compute D and E for each unselected object
D = np.zeros(len(points))
E = np.zeros(len(points))

for j in U:
    # Calculate distances to all selected medoids in S
    distances = [np.sqrt(np.sum((points[j] - points[s]) ** 2)) for s in S]
    D[j] = min(distances)  # Closest distance
    # Check if there's more than one selected medoid to get the second closest distance
    if len(distances) > 1:
        E[j] = sorted(distances)[1]  # Second closest distance
    else:
        E[j] = float('inf')  # Set to infinity if no second closest exists

print(f"Dissimilarities D: {D}")
print(f"Dissimilarities E: {E}")


Dissimilarities D: [2.82842712 2.         1.41421356 2.         0.         4.47213595
 6.32455532 5.09901951 4.         6.         4.12310563 5.
 5.38516481 6.08276253 6.70820393]
Dissimilarities E: [inf inf inf inf  0. inf inf inf inf inf inf inf inf inf inf]


In [ ]:
# Step 3: Evaluate a candidate for inclusion in the set S
# Let's choose a candidate from U for demonstration purposes
i = next(iter(U))  # Take the first element from U

C = {}
for j in U - {i}:
    Dj = D[i]
    #print(Dj)
    dji = np.sqrt(np.sum((points[j] - points[i]) ** 2))
    #print(dji)
    if Dj > dji:
        C[j] = max(Dj - dji, 0)
    else:
        C[j] = 0
    #print(C[j])

total_gain = sum(C.values())

print(f"Selected candidate index: {i}, Point: {points[i]}")
print(f"Contribution C: {C}")
print(f"Total gain from adding candidate: {total_gain}")


Selected candidate index: 0, Point: [1 1]
Contribution C: {1: 0.8284271247461903, 2: 1.4142135623730951, 3: 0.8284271247461903, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}
Total gain from adding candidate: 3.071067811865476


In [ ]:
# Step 4: Select the best candidate from U and return gains for each point
gains = []  # List to store tuples of (index, gain)

for i in U:
    # Evaluate each candidate
    #print(i)
    C = {}
    #print(C)
    for j in U - {i}:
        #print(j)
        Dj = D[i]  # Use D[candidate] as Dj
        #print(Dj)
        dji = np.sqrt(np.sum((points[j] - points[i]) ** 2))  # Distance between j and i
        #print(dji)
        if Dj > dji:
            C[j] = max(Dj - dji, 0)
        else:
            C[j] = 0
        #print(C[j])
    gain = sum(C.values())
    gains.append((i, gain))  # Store the index and gain

# Find the best candidate based on the maximum gain
best_candidate = max(gains, key=lambda x: x[1])
print(best_candidate)

print(f"All gains for candidates: {gains}")
print(f"Best candidate index: {best_candidate[0]}, Point: {points[best_candidate[0]]}, Gain: {best_candidate[1]}")

(14, 19.21561644507934)
All gains for candidates: [(0, 3.071067811865476), (1, 0.5857864376269049), (2, 0), (3, 0.5857864376269049), (5, 10.118039087878612), (6, 17.38013591456451), (7, 14.739223804878758), (8, 7.757359312880714), (9, 16.757359312880716), (10, 8.249781815351357), (11, 12.634253687263055), (12, 15.883804979045635), (13, 16.088409434073593), (14, 19.21561644507934)]
Best candidate index: 14, Point: [6 9], Gain: 19.21561644507934


In [ ]:
# Step 5: Update S and U, and recompute D and E after adding the best candidate
if best_candidate is not None:
    # Add the index of the best candidate to S
    S.add(best_candidate[0])  # Add the index
    U.remove(best_candidate[0])  # Remove the index from U

    # Recompute D for each remaining unselected point in U
    for j in U:
        distances = [np.sqrt(np.sum((points[j] - points[s]) ** 2)) for s in S]
        D[j] = min(distances)
        if len(distances) > 1:
            E[j] = sorted(distances)[1]
        else:
            E[j] = float('inf')

# Print updated S and U
print(f"Updated S (medoids): {S}")
print(f"Updated U (remaining points): {U}")
print(f"Best candidate index: {best_candidate[0]}, Point: {points[best_candidate[0]]}, Gain: {best_candidate[1]}")

Updated S (medoids): {4, 14}
Updated U (remaining points): {0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13}
Best candidate index: 14, Point: [6 9], Gain: 19.21561644507934


In [ ]:
# D and E initialization
D = np.zeros(len(points))
E = np.zeros(len(points))

# Initialize D and E for the unselected objects
for j in U:
    distances = [np.sqrt(np.sum((points[j] - points[s]) ** 2)) for s in S]
    D[j] = min(distances)
    #print(D[j]) #calculated here D[j] for all 10 points
    if len(distances) > 1:
        E[j] = sorted(distances)[1]
    else:
        E[j] = float('inf')
    #print(E[j])

# SWAP phase
while True:
    min_Tih = float('inf')
    best_pair = None

    # Step 1: Iterate over each pair (i, h)
    for i in S:
        for h in U:
            #print(h)
            K = {}  # Contributions for each j

            for j in U - {h}:
                #print(U)
                #print(j)
                #print(i)
                #print(h)
                dji = np.sqrt(np.sum((points[j] - points[i]) ** 2))
                djh = np.sqrt(np.sum((points[j] - points[h]) ** 2))
                #print(D[j])
                #print(E[j])
                #print(dji)
                #print(djh)
                if dji > D[j]:
                    if djh >= D[j]:
                        K[j] = 0
                    else:  # djh < D[j]
                        K[j] = djh - D[j]
                elif dji == D[j]:
                    if djh < E[j]:
                        K[j] = djh - D[j]
                    else:  # djh >= E[j]
                        K[j] = E[j] - D[j]
                #print(K)

            # Total result of the swap
            Tih = sum(K.values())
            #print(Tih)

            # Select the best pair (i, h) with the minimum Tih
            if Tih < min_Tih:
                min_Tih = Tih
                best_pair = (i, h)
            #print(best_pair)

    # Output results for this iteration
    if best_pair is not None and min_Tih < 0:
        i, h = best_pair

        # Perform the swap
        S.remove(i)
        S.add(h)

        # Update D and E for all objects
        for p in U.union(S):
            #print(U.union(S))
            #print(p)
            distances = [np.sqrt(np.sum((points[p] - points[s]) ** 2)) for s in S]
            #print(distances)
            D[p] = min(distances)
            #print(D[p])
            if len(distances) > 1:
                E[p] = sorted(distances)[1]
            else:
                E[p] = float('inf')
            #print(E[p])
        print(f"Swapped medoid {i} with {h}. New S: {S}")
    else:
        # No beneficial swap found, exit the loop
        break

# Final set of medoids S
print(f"Final medoids after swapping: {S}")

Swapped medoid 14 with 7. New S: {4, 7}
Final medoids after swapping: {4, 7}


In [ ]:
# Step 6: Repeat until we have k medoids
k = int(input())  # You can change this to select a different number of medoids
while len(S) < k:
    best_gain = -1
    best_candidate = None

    for i in U:
        # Evaluate each candidate
        C = {}
        for j in U - {i}:
            Dj = D[j]
            dji = np.sqrt(np.sum((points[j] - points[i]) ** 2))
            if Dj > dji:
                C[j] = max(Dj - dji, 0)
            else:
                C[j] = 0

        gain = sum(C.values())

        if gain > best_gain:
            best_gain = gain
            best_candidate = i

    if best_candidate is not None:
        S.add(best_candidate)
        U.remove(best_candidate)

        # Recompute D after updating S and U
        D = np.zeros(len(points))
        for j in U:
            D[j] = min(np.sqrt(np.sum((points[j] - points[s]) ** 2)) for s in S)

print(f"Final set of selected medoids: {S}")
print(f"Final medoid points: {[points[i] for i in S]}")


3
Final set of selected medoids: {10, 4, 7}
Final medoid points: [array([4, 7]), array([3, 3]), array([8, 2])]


In [ ]:
import numpy as np

# Given points
points = np.array([
    (1, 1), (3, 1), (2, 2), (1, 3), (3, 3), (7, 1), (9, 1), (8, 2), (7, 3),
    (9, 3), (4, 7), (6, 7), (5, 8), (4, 9), (6, 9)
])

# Function to calculate Euclidean distance
def euclidean_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))

# Function to find k medoids
def find_k_medoids(points, k):
    # Step 1: Initialize S by finding the first medoid with the minimal sum of distances
    min_index = 0
    min_distance_sum = float('inf')

    for i in range(len(points)):
        distance_sum = sum(euclidean_distance(points[i], points[j]) for j in range(len(points)) if j != i)
        if distance_sum < min_distance_sum:
            min_distance_sum = distance_sum
            min_index = i

    # Initialize S with the first medoid and U with remaining points
    S = {min_index}
    U = {i for i in range(len(points)) if i != min_index}

    print(f"Initial medoid index: {min_index}, Medoid point: {points[min_index]}")
    print('-' * 50)


    # Iteratively find the next medoids until we have k medoids
    while len(S) < k:
        D = np.zeros(len(points))  # Dissimilarity to closest selected medoid
        E = np.zeros(len(points))  # Dissimilarity to second closest selected medoid

        # Step 2: Compute D and E for each unselected object
        for j in U:
            distances = [euclidean_distance(points[j], points[s]) for s in S]
            D[j] = min(distances)  # Closest distance
            E[j] = sorted(distances)[1] if len(distances) > 1 else float('inf')  # Second closest distance

        #print(f"Dissimilarities D: {D}")
        #print(f"Dissimilarities E: {E}")

        # Step 3: Select the best candidate from U and return gains for each point
        gains = []  # List to store tuples of (index, gain)

        for i in U:
            C = {}
            for j in U - {i}:
                Dj = D[i]  # Use D[candidate] as Dj
                dji = euclidean_distance(points[j], points[i])  # Distance between j and i
                if Dj > dji:
                    C[j] = max(Dj - dji, 0)
                else:
                    C[j] = 0
            gain = sum(C.values())
            gains.append((i, gain))  # Store the index and gain

        # Find the best candidate based on the maximum gain
        best_candidate = max(gains, key=lambda x: x[1])
        print(f"All gains for candidates: {gains}")
        print(f"Best candidate index: {best_candidate[0]}, Point: {points[best_candidate[0]]}, Gain: {best_candidate[1]}")
        print('-' * 50)


        # Step 5: Update S and U, and recompute D and E after adding the best candidate
        if best_candidate is not None:
            # Add the index of the best candidate to S
            S.add(best_candidate[0])  # Add the index
            U.remove(best_candidate[0])  # Remove the index from U

            # Recompute D and E for each remaining unselected point in U
            for j in U:
                distances = [euclidean_distance(points[j], points[s]) for s in S]
                D[j] = min(distances)
                E[j] = sorted(distances)[1] if len(distances) > 1 else float('inf')

        # Print updated S and U
        print(f"Updated S (medoids): {S}")
        print(f"Updated U (remaining points): {U}")
        print('-' * 50)

    # Now perform the SWAP phase to refine the medoids
    # D and E initialization
    D = np.zeros(len(points))
    E = np.zeros(len(points))

    # Initialize D and E for the unselected objects
    for j in U:
        distances = [euclidean_distance(points[j], points[s]) for s in S]
        D[j] = min(distances)
        E[j] = sorted(distances)[1] if len(distances) > 1 else float('inf')

    # SWAP phase
    while True:
        min_Tih = float('inf')
        best_pair = None

        # Step 1: Iterate over each pair (i, h)
        for i in S:
            for h in U:
                K = {}  # Contributions for each j

                for j in U - {h}:
                    dji = euclidean_distance(points[j], points[i])
                    djh = euclidean_distance(points[j], points[h])

                    if dji > D[j]:
                        if djh >= D[j]:
                            K[j] = 0
                        else:  # djh < D[j]
                            K[j] = djh - D[j]
                    elif dji == D[j]:
                        if djh < E[j]:
                            K[j] = djh - D[j]
                        else:  # djh >= E[j]
                            K[j] = E[j] - D[j]

                # Total result of the swap
                Tih = sum(K.values())
                print(Tih)
                # Select the best pair (i, h) with the minimum Tih
                if Tih < min_Tih:
                    min_Tih = Tih
                    best_pair = (i, h)

        # Output results for this iteration
        if best_pair is not None and min_Tih < 0:
            i, h = best_pair

            # Perform the swap
            S.remove(i)
            S.add(h)

            # Update D and E for all objects
            for p in U.union(S):
                distances = [euclidean_distance(points[p], points[s]) for s in S]
                D[p] = min(distances)
                E[p] = sorted(distances)[1] if len(distances) > 1 else float('inf')

            print(f"Swapped medoid {i} with {h}. New S: {S}")
            print('-' * 50)
        else:
            # No beneficial swap found, exit the loop
            break

    # Final set of medoids S
    print(f"Final medoids after swapping: {S}")
    print('-' * 50)
    return S  # Return the selected medoid indices

# Specify the number of medoids k
k = int(input())  # Change this value to your desired number of medoids
selected_medoids = find_k_medoids(points, k)
print(f"Final set of medoid indices: {selected_medoids}")
print(f"Medoid points: {[points[i] for i in selected_medoids]}")
